In [2]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [5]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=19)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [6]:
to_score = [d[:100] for d in ds][:10]
scores = decomp.score(
	to_score
)

DS (3, 278, 512) 278
(278, 3, 512) [8, 57, 89]


In [16]:
tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)['input_ids']] for d in to_score]
tokens[0]

['Con',
 'vention',
 'ally',
 ',',
 ' a',
 ' scheme',
 ' that',
 ' a',
 ' value',
 ' of',
 ' plural',
 ' bits',
 ' is',
 ' stored',
 ' in',
 ' a',
 ' single',
 ' memory',
 ' cell',
 ' transistor',
 ' is']

In [26]:
from circuitsvis.utils.render import RenderedHTML, render
import matplotlib

# visualize_text(model, tokens, scores)
html = render(
        "TextNeuronActivations",
        tokens=tokens,
        activations=[[[s]] for s in scores],
        firstDimensionName="Layer",
        secondDimensionName="Neuron",
        firstDimensionLabels=None,
				secondDimensionLabels=second_dimension_labels
    )

In [25]:
html